## Importing the Necessary Library


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
from datetime import datetime
import pytz


In [2]:
print(os.getcwd())


/Users/abhishekkandel/Desktop/LOG Analysis


In [3]:
os.chdir('/Users/abhishekkandel/Desktop/LOG Analysis/')
print(os.getcwd())
client_df = pd.read_csv('client_hostname.csv', sep='\t')
client_df.head()

/Users/abhishekkandel/Desktop/LOG Analysis


,"client,hostname,alias_list,address_list"
0,"5.123.144.95,5.123.144.95,[Errno 1] Unknown host,"
1,"5.122.76.187,5.122.76.187,[Errno 1] Unknown host,"
2,"5.215.249.99,5.215.249.99,[Errno 1] Unknown host,"
3,"31.56.102.211,31-56-102-211.shatel.ir,['211.10..."
4,"5.123.166.223,5.123.166.223,[Errno 1] Unknown ..."


## Data Pre-processing

The log file comprises 3.3GB of web server logs extracted from zanbil.ir, an Iranian ecommerce platform, offering a comprehensive view of user interactions, crawler activities, and business trends. This log file, compiled by Zaker and Farzin in 2019, is available via Harvard Dataverse for research and analytical purposes.

### Loading the log file into a dataframe

I extracted relevant information such as client IP, user ID, timestamp, HTTP method, request, status code, size, referer, and user agent from each log line.

In [ ]:
import pandas as pd
import re
log_file_path = './access.log'
# Define the regex pattern to extract information from log lines
regex_pattern = r'^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[\w:/]+\s[+\-]\d{4})\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-) "(?P<referer>[^"]*)" "(?P<user_agent>[^"]*)"'

# Define the column names
columns = ['client', 'userid', 'datetime', 'method', 'request', 'status', 'size', 'referer', 'user_agent']

# Read the log file in chunks of 1000 lines
log_data = []
chunk_size = 1000
with open(log_file_path, 'r') as file:
    while True:
        lines = list(file.readline() for _ in range(chunk_size))
        if not any(lines):
            break
        for line in lines:
            if not line.strip():
                continue
            match = re.match(regex_pattern, line)
            if match:
                log_data.append({
                    'client': match.group('client'),
                    'userid': match.group('userid'),
                    'datetime': match.group('datetime'),
                    'method': match.group('method'),
                    'request': match.group('request'),
                    'status': match.group('status'),
                    'size': match.group('size'),
                    'referer': match.group('referer'),
                    'user_agent': match.group('user_agent')
                })
            else:
                print("Error: Line does not match regex pattern:", line)

# Create DataFrame from the list of dictionaries
logs_df = pd.DataFrame(log_data, columns=columns)
# print total processed lines 
print("Total processed lines: ", len(logs_df))


In [5]:
# Diplaying the first 5 rows of the dataframe
logs_df.head()


,client,userid,datetime,method,request,status,size,referer,user_agent
0,54.36.149.41,-,22/Jan/2019:03:56:14 +0330,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...
1,31.56.96.51,-,22/Jan/2019:03:56:16 +0330,GET,/image/60844/productModel/200x200,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
2,31.56.96.51,-,22/Jan/2019:03:56:16 +0330,GET,/image/61474/productModel/200x200,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
3,40.77.167.129,-,22/Jan/2019:03:56:17 +0330,GET,/image/14925/productModel/100x100,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
4,91.99.72.15,-,22/Jan/2019:03:56:17 +0330,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...


#### Understanding and processing the dataset¶


In [6]:
# Checking the overview of the dataframe
logs_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10364865 entries, 0 to 10364864
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   client      object
 1   userid      object
 2   datetime    object
 3   method      object
 4   request     object
 5   status      object
 6   size        object
 7   referer     object
 8   user_agent  object
dtypes: object(9)
memory usage: 711.7+ MB


In [7]:
 # Function to parse the datetime (from the class session practice exercise)
def parse_datetime(x):
    '''
    Parses datetime with timezone formatted as:
        `[day/month/year:hour:minute:second zone]`

    Example:
        `>>> parse_datetime('13/Nov/2015:11:45:42 +0000')`
        `datetime.datetime(2015, 11, 3, 11, 45, 4, tzinfo=<UTC>)`

    Due to problems parsing the timezone (`%z`) with `datetime.strptime`, the
    timezone will be obtained using the `pytz` library.
    '''
    try:
        dt = datetime.strptime(x[1:-7], '%d/%b/%Y:%H:%M:%S')
        dt_tz = int(x[-6:-3])*60+int(x[-3:-1])
        return dt.replace(tzinfo=pytz.FixedOffset(dt_tz))
    except ValueError:
        return '-'

In [8]:
logs_df['status'] = logs_df['status'].astype(int) # Convert the status column to integer
logs_df['size'] = logs_df['size'].astype(int) # Convert the size column to integer
logs_df['datetime'] = logs_df['datetime'].apply(parse_datetime) # Apply the parse_datetime function to the datetime column

In [9]:
logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10364865 entries, 0 to 10364864
Data columns (total 9 columns):
 #   Column      Dtype                               
---  ------      -----                               
 0   client      object                              
 1   userid      object                              
 2   datetime    datetime64[ns, pytz.FixedOffset(33)]
 3   method      object                              
 4   request     object                              
 5   status      int64                               
 6   size        int64                               
 7   referer     object                              
 8   user_agent  object                              
dtypes: datetime64[ns, pytz.FixedOffset(33)](1), int64(2), object(6)
memory usage: 711.7+ MB


In [10]:
logs_df.head()

,client,userid,datetime,method,request,status,size,referer,user_agent
0,54.36.149.41,-,2019-01-02 03:56:01+00:33,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...
1,31.56.96.51,-,2019-01-02 03:56:01+00:33,GET,/image/60844/productModel/200x200,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
2,31.56.96.51,-,2019-01-02 03:56:01+00:33,GET,/image/61474/productModel/200x200,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
3,40.77.167.129,-,2019-01-02 03:56:01+00:33,GET,/image/14925/productModel/100x100,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
4,91.99.72.15,-,2019-01-02 03:56:01+00:33,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...


In [11]:
# Dropping the userid column
# This because it has one unique value which is just a hyphen

users = logs_df['userid'].unique()
print(users)


['-' 'admin']


In [12]:
# Show where uderid is not a hyphen
logs_df[logs_df['userid'] != '-']


,client,userid,datetime,method,request,status,size,referer,user_agent
7895845,124.235.138.251,admin,2019-01-05 21:28:02+00:33,GET,/,301,178,-,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
7895859,124.235.138.251,admin,2019-01-05 21:28:02+00:33,GET,/,301,178,-,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....


In [13]:
logs_df.drop(columns=['userid'], inplace=True)


## Dropping Duplicates

There were duplicates which are not adding value to the analysis.

In [14]:
# Count duplicates in the dataframe
duplicate_count = logs_df.duplicated().sum()

# Display the count of duplicates
print("Number of duplicates:", duplicate_count)

Number of duplicates: 313398


In [15]:
# Drop the duplicates
logs_df = logs_df.drop_duplicates()

In [16]:
# Diplaying the first 5 rows of the dataframe
logs_df.head()

,client,datetime,method,request,status,size,referer,user_agent
0,54.36.149.41,2019-01-02 03:56:01+00:33,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...
1,31.56.96.51,2019-01-02 03:56:01+00:33,GET,/image/60844/productModel/200x200,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
2,31.56.96.51,2019-01-02 03:56:01+00:33,GET,/image/61474/productModel/200x200,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
3,40.77.167.129,2019-01-02 03:56:01+00:33,GET,/image/14925/productModel/100x100,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
4,91.99.72.15,2019-01-02 03:56:01+00:33,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...


In [17]:
# Exporting the cleaned dataframe to a csv file for further analysis
logs_df.to_csv('cleaned_access_logs.csv', index=False)

## Merging the Hostname and Client

To enhance our dataset, we will merge the `client_df` dataframe, which contains client IP addresses and their corresponding hostnames, with the `logs_df` dataframe. This will allow us to associate each log entry with its respective hostname, providing more context for our analysis.

In [18]:
# Read the client_hostname CSV file
client_hostname_path = './client_hostname.csv'
client_hostname_df = pd.read_csv(client_hostname_path)

# Merge logs_df with client_hostname_df on 'client'
logs_df = logs_df.merge(client_hostname_df, on='client', how='left')

In [19]:
logs_df.head()


,client,datetime,method,request,status,size,referer,user_agent,hostname,alias_list,address_list
0,54.36.149.41,2019-01-02 03:56:01+00:33,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...,54.36.149.41,[Errno 1] Unknown host,NaN
1,31.56.96.51,2019-01-02 03:56:01+00:33,GET,/image/60844/productModel/200x200,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,31.56.96.51,[Errno 1] Unknown host,NaN
2,31.56.96.51,2019-01-02 03:56:01+00:33,GET,/image/61474/productModel/200x200,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,31.56.96.51,[Errno 1] Unknown host,NaN
3,40.77.167.129,2019-01-02 03:56:01+00:33,GET,/image/14925/productModel/100x100,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,40.77.167.129,[Errno 1] Unknown host,NaN
4,91.99.72.15,2019-01-02 03:56:01+00:33,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...,91.99.72.15,[Errno 1] Unknown host,NaN


In [20]:
logs_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10051467 entries, 0 to 10051466
Data columns (total 11 columns):
 #   Column        Dtype                               
---  ------        -----                               
 0   client        object                              
 1   datetime      datetime64[ns, pytz.FixedOffset(33)]
 2   method        object                              
 3   request       object                              
 4   status        int64                               
 5   size          int64                               
 6   referer       object                              
 7   user_agent    object                              
 8   hostname      object                              
 9   alias_list    object                              
 10  address_list  object                              
dtypes: datetime64[ns, pytz.FixedOffset(33)](1), int64(2), object(8)
memory usage: 843.6+ MB


In [22]:
logs_df.head()

,client,datetime,method,request,status,size,referer,user_agent,hostname,alias_list,address_list
0,54.36.149.41,2019-01-02 03:56:01+00:33,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...,54.36.149.41,[Errno 1] Unknown host,NaN
1,31.56.96.51,2019-01-02 03:56:01+00:33,GET,/image/60844/productModel/200x200,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,31.56.96.51,[Errno 1] Unknown host,NaN
2,31.56.96.51,2019-01-02 03:56:01+00:33,GET,/image/61474/productModel/200x200,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,31.56.96.51,[Errno 1] Unknown host,NaN
3,40.77.167.129,2019-01-02 03:56:01+00:33,GET,/image/14925/productModel/100x100,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,40.77.167.129,[Errno 1] Unknown host,NaN
4,91.99.72.15,2019-01-02 03:56:01+00:33,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...,91.99.72.15,[Errno 1] Unknown host,NaN


In [23]:
# Replace the vlaue'- ' with 'unknown' in the 'hostname' column
logs_df['hostname'] = logs_df['hostname'].replace('-', 'unknown')

#raplaceing the value '-' in referer column with 'direct'
logs_df['referer'] = logs_df['referer'].replace('-', 'direct')


In [ ]:
logs_df.head()

,client,datetime,method,request,status,size,referer,user_agent,hostname,alias_list,address_list
0,54.36.149.41,2019-01-02 03:56:01+00:33,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,200,30577,direct,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...,54.36.149.41,[Errno 1] Unknown host,NaN
1,31.56.96.51,2019-01-02 03:56:01+00:33,GET,/image/60844/productModel/200x200,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,31.56.96.51,[Errno 1] Unknown host,NaN
2,31.56.96.51,2019-01-02 03:56:01+00:33,GET,/image/61474/productModel/200x200,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,31.56.96.51,[Errno 1] Unknown host,NaN
3,40.77.167.129,2019-01-02 03:56:01+00:33,GET,/image/14925/productModel/100x100,200,1696,direct,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,40.77.167.129,[Errno 1] Unknown host,NaN
4,91.99.72.15,2019-01-02 03:56:01+00:33,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,200,41483,direct,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...,91.99.72.15,[Errno 1] Unknown host,NaN


In [25]:
#  Exporting the cleaned dataframe to a csv file with the hostname and referer columns cleaned
logs_df.to_csv('cleaned_access_logs_with_hostnames.csv', index=False)

